In [8]:
import cv2
import torch
from transformers import ViTImageProcessor, ViTForImageClassification

# Initialize the image processor and model
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTForImageClassification.from_pretrained('ChrisGuarino/model')  # Replace with your model
model.eval()

# Define your class labels
class_labels = ['Prim', 'Rupe', 'No Cat']  # Replace with your actual labels

# Start the webcam
cap = cv2.VideoCapture(0)

#Confidence Threshold
confidence_threshold = .8  # Define a threshold

while True:
    ret, frame = cap.read()

    if ret:
        # Preprocess the frame
        frame_resized = cv2.resize(frame, (400, 400))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        inputs = processor(images=frame_rgb, return_tensors="pt")

        # Get predicti?ons
        with torch.no_grad():
            predictions = model(**inputs).logits

            # Convert predictions to probabilities and get the highest probability class
            probabilities = torch.nn.functional.softmax(predictions, dim=-1)
            confidences, predicted_class_idx = torch.max(probabilities, dim=-1)
            predicted_class = class_labels[predicted_class_idx]#Something with +1 to shift the labels if we add a No Cat label
        print(probabilities)

        # Check if confidence is above the threshold
        if confidences.item() < confidence_threshold:
            label = 'No Cat'
            confidence = 0
        else:
            label = class_labels[predicted_class_idx.item()]  # +1 to account for 'No Cat'
            confidence = confidences.item()

        # Prepare the display text
        display_text = f'{label} ({confidence:.2f})'

        # Display the prediction on the frame
        cv2.putText(frame_resized, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the resulting frame
        cv2.imshow('Frame', frame_resized)

        # Break the loop with 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Release the capture when done
cap.release()
cv2.destroyAllWindows()


tensor([[0.0204, 0.0198, 0.9598]])
tensor([[0.0204, 0.0202, 0.9594]])
tensor([[0.0200, 0.0199, 0.9601]])
tensor([[0.0191, 0.0193, 0.9616]])
tensor([[0.0196, 0.0196, 0.9608]])
tensor([[0.0197, 0.0198, 0.9605]])
tensor([[0.0195, 0.0202, 0.9603]])
tensor([[0.0199, 0.0210, 0.9591]])
tensor([[0.0200, 0.0216, 0.9584]])
tensor([[0.0199, 0.0213, 0.9588]])
tensor([[0.0202, 0.0220, 0.9577]])
tensor([[0.0199, 0.0214, 0.9587]])
tensor([[0.0198, 0.0213, 0.9589]])
tensor([[0.0199, 0.0217, 0.9584]])
tensor([[0.0200, 0.0218, 0.9582]])
tensor([[0.0198, 0.0214, 0.9588]])
tensor([[0.0202, 0.0221, 0.9577]])
tensor([[0.0197, 0.0204, 0.9600]])
tensor([[0.0201, 0.0209, 0.9589]])
tensor([[0.0200, 0.0205, 0.9595]])
tensor([[0.0196, 0.0197, 0.9607]])
tensor([[0.0203, 0.0189, 0.9608]])
tensor([[0.0221, 0.0210, 0.9569]])
tensor([[0.0201, 0.0194, 0.9605]])
tensor([[0.0212, 0.0201, 0.9587]])
tensor([[0.0217, 0.0211, 0.9571]])
tensor([[0.0210, 0.0205, 0.9585]])
tensor([[0.0209, 0.0218, 0.9573]])
tensor([[0.0211, 0.0